# ch07-종합을 위한 그룹화, 필터링, 그리고 변환

- 데이터 분석 중 가장 기본적인 과제는 각 그룹에 대한 계산을 수행하기 전에 데이터를 독립적인 그룹으로 분할하는 것이다.
- 분할-적용-병합 split-apply-combine 으로 불린다.
- groupby() 메서드의 결과는 groupby 객체다.
- groupby 객체가 생성될 때 pandas가 하는 일을 별로 없고 단지 주어진 조건의 그룹화가 가능한지 확인만 한다.
- 제대로 사용하려면 groupby 객체에 메서드를 체인시켜야 한다.

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

## 8.체중 감량 내기를 통한 변환

- 두 명의 체중 감량을 4개월간 추적해 승자를 결정한다.
- 각 달의 마지막에 그 달에 가장 체중 강량 비율이 큰 사람이 승자가 된다.
- 체중 감량을 추적하기 위해 데이터를 월별, 사람별로 그룹화하고
- transform() 메서드를 사용하 시작주 부터 각 주마다의 체중 감량 비율을 찾아본다.

- weight_loss 데이터셋을 읽는다.

In [2]:
weight_loss = pd.read_csv('../data/weight_loss.csv')

In [3]:
weight_loss.head()

,Name,Month,Week,Weight
0,Bob,Jan,Week 1,291
1,Amy,Jan,Week 1,197
2,Bob,Jan,Week 2,288
3,Amy,Jan,Week 2,189
4,Bob,Jan,Week 3,283


In [4]:
weight_loss.shape

(32, 4)

In [5]:
weight_loss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 4 columns):
Name      32 non-null object
Month     32 non-null object
Week      32 non-null object
Weight    32 non-null int64
dtypes: int64(1), object(3)
memory usage: 680.0+ bytes


In [6]:
weight_loss.query('Month == "Jan"')

,Name,Month,Week,Weight
0,Bob,Jan,Week 1,291
1,Amy,Jan,Week 1,197
2,Bob,Jan,Week 2,288
3,Amy,Jan,Week 2,189
4,Bob,Jan,Week 3,283
5,Amy,Jan,Week 3,189
6,Bob,Jan,Week 4,283
7,Amy,Jan,Week 4,190


- 각 달의 승자를 결정하기 위해서는 각 달의 첫 번째 주와 마지막 주의 체중 변화만 계산하면 된다.
- 주별로 정보를 갱신하고자 한다면 매주마다 그 달의 첫 번째 주부터 체중 감량을 계산해볼 수 있다.

In [7]:
def find_perc_loss(s):
    return (s - s.iloc[0]) / s.iloc[0]

In [8]:
bob_jan = weight_loss.query('Name == "Bob" and Month == "Jan"')

In [9]:
bob_jan

,Name,Month,Week,Weight
0,Bob,Jan,Week 1,291
2,Bob,Jan,Week 2,288
4,Bob,Jan,Week 3,283
6,Bob,Jan,Week 4,283


In [10]:
find_perc_loss(bob_jan['Weight'])

0    0.000000
2   -0.010309
4   -0.027491
6   -0.027491
Name: Weight, dtype: float64

- 이 함수를 사람과 주에 대한 모든 단일 조합에 대해 적용하면 각 달의 첫 주와 대비한 주별 체중 감량값을 얻을 수 있다.
- 데이터를 Name과 Month에 따라 그룹화하고 transform() 메서드를 사용해 이 커스컴 함수를 적용한다.

In [11]:
pcnt_loss = weight_loss.groupby(['Name', 'Month'])['Weight']\
                       .transform(find_perc_loss)

In [12]:
pcnt_loss.shape

(32,)

In [13]:
pcnt_loss.head(8)

0    0.000000
1    0.000000
2   -0.010309
3   -0.040609
4   -0.027491
5   -0.040609
6   -0.027491
7   -0.035533
Name: Weight, dtype: float64

- transform() 메서드는 호출하는 DataFrame과 동일한 행 개수를 갖는 객체를 반환해야만 한다.
- 이 결과를 최초 DataFrame에 새로운 col로 추가해보자.
- 출력을 단순화하기 위해 bob의 처음 두 달 데이터만 선택한다.

In [14]:
weight_loss['Perc Weight Loss'] = pcnt_loss.round(3)

In [15]:
weight_loss.query('Name=="Bob" and Month in ["Jan", "Feb"]')

,Name,Month,Week,Weight,Perc Weight Loss
0,Bob,Jan,Week 1,291,0.000
2,Bob,Jan,Week 2,288,-0.010
4,Bob,Jan,Week 3,283,-0.027
6,Bob,Jan,Week 4,283,-0.027
8,Bob,Feb,Week 1,283,0.000
10,Bob,Feb,Week 2,275,-0.028
12,Bob,Feb,Week 3,268,-0.053
14,Bob,Feb,Week 4,268,-0.053


- 매달 체중 감량 비율은 리셋되야 한다.
- 중요한 것은 오직 마지막 주의 결과이므로 4주째를 선택한다.

In [16]:
week4 = weight_loss.query('Week == "Week 4"')

In [17]:
week4

,Name,Month,Week,Weight,Perc Weight Loss
6,Bob,Jan,Week 4,283,-0.027
7,Amy,Jan,Week 4,190,-0.036
14,Bob,Feb,Week 4,268,-0.053
15,Amy,Feb,Week 4,173,-0.089
22,Bob,Mar,Week 4,261,-0.026
23,Amy,Mar,Week 4,170,-0.017
30,Bob,Apr,Week 4,250,-0.042
31,Amy,Apr,Week 4,161,-0.053


- 4주째만 선택했지만 각 월의 승자를 자동으로 찾아내지는 못한다.
- pivot 메서드로 데이터를 재구성해 체중 감량 비율이 각 월별로 인접해 나타나도록 수정하자.

In [18]:
winner = week4.pivot(index='Month', columns='Name',
                     values='Perc Weight Loss')

In [19]:
winner

Name,Amy,Bob
Month,,
Apr,-0.053,-0.042
Feb,-0.089,-0.053
Jan,-0.036,-0.027
Mar,-0.017,-0.026


- 이 출력은 각 월별로 누가 승자인지 확연히 보여주지만, 자동화를 위해서는 여전히 몇 단계 작업을 더 거쳐야 한다.
- Numpy에는 where라는 이름의 벡터화된 if-then-else 함수가 있어서 Series나 불리언 배열을 다른 값으로 매핑할 수 있다.
- 승자 이름을 담을 새로운 col을 생성하고 그 달의 승자의 감량 비율을 부각시켜 보자.

In [20]:
winner['Winner'] = np.where(winner['Amy'] < winner['Bob'], 'Amy', 'Bob')

In [21]:
winner.style.highlight_min(axis=1)

Name,Amy,Bob,Winner
Month,,,
Apr,-0.053,-0.042,Amy
Feb,-0.089,-0.053,Amy
Jan,-0.036,-0.027,Amy
Mar,-0.017,-0.026,Bob


- 마지막으로 value_counts() 메서드로 승리한 월 개수를 세어 최종 점수를 반환한다.

In [22]:
winner.Winner.value_counts()

Amy    3
Bob    1
Name: Winner, dtype: int64

- 결과를 도출하는 동안 불리언 인덱싱 대신 query() 메서드를 사용해 데이터를 필터링 했다.
- query() 메서드를 사용하면 불리언 인덱싱보다 가독성이 좋아진다.

- 각 사람의 월별 체중 감량 비율을 찾기 위해서는 각 주의 체중 감소를 월 초와 비교해야 한다.
- 이 상황에서는 groupby의 transform() 메서드가 가장 적합하다.
- transform() 메서드는 함수를 매개변수 중 하나로 받아들인다.
- 이 함수는 묵시적으로 그룹화되지 않은 각 col에 전달된다.
- 혹은 이번 예제처럼 Weight col에서 한 것처럼 인덱스 연산자에서 지정할 특정 col에 전달된다.
- 이 메서드는 전달된 그룹과 동일한 길이의 값을 반환하고, 그렇지 않으면 오류가 발생한다.
- 원시 DataFrame 으로부터의 모든 값이 변환된다.
- 어떠한 aggregation이나 filtering도 일어나지 않는다.

- pivot 함수는 데이터셋을 한 col의 고유한 값을 중심축으로 새로운 col 이름으로 만들어준다.
- index 매개변수를 사용하면 pivot으로 하기를 원하지 않는 열을 지정할 수 있다.
- values 매개변수에 전달된 col들은 그 index와 columns 매개변수의 각 고유한 조합에 대해 연속적으로 쌓인다.

- pivot() 메서드는 index와 columns 매개변수의 고유한 조합이 단 한 번만 나타날 때만 작동한다.
- 고유한 조합이 둘 이상 존재하면 오류가 발생한다.
- 그런 경우에는 pivot_table을 사용해 복수값을 한꺼번에 종합할 수 있다.

- Numpy의 where 함수의 첫 번째 인수는 불리언 Series를 생성할 조건이다.

In [23]:
winner.style.highlight_min(axis=1)

Name,Amy,Bob,Winner
Month,,,
Apr,-0.053,-0.042,Amy
Feb,-0.089,-0.053,Amy
Jan,-0.036,-0.027,Amy
Mar,-0.017,-0.026,Bob


- pandas는 Month를 알파벳순으로 정렬한다.
- 이것은 월의 데이터 형식을 범주형으로 바꾸면 해결할 수 있다.
- 범주형 변수는 각 열의 모든 값을 정수로 매핑한다.

In [24]:
week4a = week4.copy()

In [25]:
month_chron = week4a['Month'].unique() # or use drop_ducplicate

In [26]:
month_chron

array(['Jan', 'Feb', 'Mar', 'Apr'], dtype=object)

In [27]:
week4a['Month'] = pd.Categorical(week4a['Month'],
                                 categories=month_chron,
                                 ordered=True)

In [28]:
week4a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8 entries, 6 to 31
Data columns (total 5 columns):
Name                8 non-null object
Month               8 non-null category
Week                8 non-null object
Weight              8 non-null int64
Perc Weight Loss    8 non-null float64
dtypes: category(1), float64(1), int64(1), object(2)
memory usage: 376.0+ bytes


In [29]:
week4a.pivot(index='Month',
             columns='Name',
             values='Perc Weight Loss')

Name,Amy,Bob
Month,,
Jan,-0.036,-0.027
Feb,-0.089,-0.053
Mar,-0.017,-0.026
Apr,-0.053,-0.042


- Month col을 변환하기 위해 Categorical 생성자를 사용한다.
- 원래 col을 Series에 전달하고 categories 매개변수에 모든 범주를 원하는 고유의 순서로 전달한다.
- 일반적으로 객체 대이터 형식의 col을 알파벳순이 아닌 다른 순서로 정렬하기 위해 category형으로 변환한다.

## 9.apply를 사용해 주별 SAT 가중 평균 구하기

- grouopby 객체는 각 그룹별로 계산을 수행하기 위해 함수를 받아들일 수 있는 네 가지 메서드를 갖고 있다.
- 이 네 가지 메서드는 agg, filter, transform, apply다.
- 처음 세 가지 메서드는 각각 함수가 반환해야하는 특정 출력이 정해져 있다.
- agg는 스칼라 값, filter는 불리언, transform은 전달된 그룹과 동일한 길이의 Series를 반환한다.
- apply() 메서드는 스칼라, Series, DataFrmae도 반환할 수 있어 아주 유연하다.
- 그룹별로 한 번만 호출되므로 각 그룹화되지 않은 열에 오직 한 번만 호출되는 transform이나 agg와 대조된다.
- apply 메서드는 동시에 복수 개의 열에 대해 연산할 때 단일 객체를 반환하는 기능도 있다.

- 대학 데이터셋으로부터 수학과 구술 점수의 가중 평균을 계산해보자.
- 대학 데이터셋을 읽은 후 UGDS, SATMTMID, SATVRMID col 중 누락값이 있는 행은 삭제한다.

In [30]:
college = pd.read_csv('../data/college.csv')

In [31]:
subset = ['UGDS', 'SATMTMID', 'SATVRMID']

In [32]:
college2 = college.dropna(subset=subset)

- dropna() 메서드의 subset 매개변수는 누락값을 살펴보는 col의 범위를 제한한다.

In [33]:
college.shape

(7535, 27)

In [34]:
college2.shape

(1184, 27)

- SAT 수학 점수만의 가중 평균을 계산하기 위한 사용자 정의 함수를 생성한다.

In [35]:
def weighted_math_average(df):
    weighted_math = df['UGDS'] * df['SATMTMID']
    return int(weighted_math.sum() / df['UGDS'].sum())

In [36]:
college2.groupby('STABBR').apply(weighted_math_average).head()

STABBR
AK    503
AL    536
AR    529
AZ    569
CA    564
dtype: int64

- 같은 함수를 agg() 메서드에 전달하면 어떤 결과가 나올까?

In [37]:
college2.groupby('STABBR').agg(weighted_math_average).head()

,INSTNM,CITY,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
STABBR,,,,,,,,,,,,,,,,,,,,,
AK,503,503,503,503,503,503,503,503,503,503,...,503,503,503,503,503,503,503,503,503,503
AL,536,536,536,536,536,536,536,536,536,536,...,536,536,536,536,536,536,536,536,536,536
AR,529,529,529,529,529,529,529,529,529,529,...,529,529,529,529,529,529,529,529,529,529
AZ,569,569,569,569,569,569,569,569,569,569,...,569,569,569,569,569,569,569,569,569,569
CA,564,564,564,564,564,564,564,564,564,564,...,564,564,564,564,564,564,564,564,564,564


- weighted_math_average 함수는 DataFrame에서 aggregation되지 않은 각 col에 대해 적용됐다.
- col을 SATMTMID로만 제한하려 한다면 UGDS에 접근할 수 없으므로 오류가 발생한다.

- apply가 가진 좋은 점은 Series를 반환해 복수 개의 새로운 열을 생성할 수 있다는 것이다.
- 반환된 Series의 인덱스는 새 col의 이름이 된다.

In [38]:
from collections import OrderedDict
def weighted_average(df):
    data = OrderedDict()
    weight_m = df['UGDS'] * df['SATMTMID']
    weight_v = df['UGDS'] * df['SATVRMID']
    wm_avg = weight_m.sum() / df['UGDS'].sum()
    wv_avg = weight_v.sum() / df['UGDS'].sum()
    
    data['weighted_math_avg'] = wm_avg
    data['weighted_verbal_avg'] = wv_avg
    data['math_avg'] = df['SATMTMID'].mean()
    data['verbal_avg'] = df['SATVRMID'].mean()
    data['count'] = len(df)
    return pd.Series(data, dtype='int')

In [39]:
college2.groupby('STABBR').apply(weighted_average).head(10)

,weighted_math_avg,weighted_verbal_avg,math_avg,verbal_avg,count
STABBR,,,,,
AK,503,555,503,555,1
AL,536,533,504,508,21
AR,529,504,515,491,16
AZ,569,557,536,538,6
CA,564,539,562,549,72
CO,553,547,540,537,14
CT,545,533,522,517,14
DC,621,623,588,589,6
DE,569,553,495,486,3


- apply로 복수 개의 col을 상성하기 위해서는 weighted_average 함수는 Series를 반환해야 한다.
- 결과 DataFrame에서는 인덱스 값이 col의 이름으로 사용되었다.

In [40]:
# from scipy.stats import gmean, hmean
# def calculate_means(df):
#     df_menas = pd.DataFrame(index=['Arithmetic', 'Weighted', 'Geometric', 'Harmonic'])
#     cols = ['SATMTMID', 'SATVRMID']
#     for col in cols:
#         arithmetic = df[col].mean()
#         weighted = np.average(df[col], weights=df['UGDS'])
#         geometric = gmean(df[col])
#         harmonic = hmean(df[col])
#         df_means[col] = [arithmetic, weighted, geometric. harmonic]

#     df_means['count'] = len(df)
#     return df_menas.astype(int)

In [41]:
# college2.groupby('STABBR').apply(calculate_means).head()

## 10.연속 변수에 대한 그룹화

- pandas에서 그룹화할 때는 대개 이산값이 반복되는 열을 사용한다.
- 연속 수치를 가진 열은 대개 반복되는 값이 없으므로 일반적으로 그룹화에 사용되지 않는다.
- 그러나 연속값을 가진 열을 각 값을 어느 점위에 포함한다든지, 반올림한다든지, 다른 매핑을 사용해 이산값을 가진 열로 변환하면 그룹화를 수행할 수 있다.

- 비행 데이터셋을 이용하여 운항 거리에 대한 분포를 조사한다.
- 예를 들면 500마일과 1000마일 사이의 거리를 가장 많이 비행하는 항공사를 찾을 수 있다.
- pandas cut 함수를 사용해 각 비행 거리를 이산값으로 바꾼다.

In [42]:
flights = pd.read_csv('../data/flights.csv')

In [43]:
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


- 거리의 범위에 대한 항공사의 분포를 찾으려면 DIST col에 있는 값을 이산값의 범위에 매핑해야 한다.
- pandas cut 함수를 사용해 데이터를 5개의 구간으로 분할한다.

In [44]:
bins = [-np.inf, 200, 500, 1000, 2000, np.inf]

In [45]:
cuts = pd.cut(flights['DIST'], bins=bins)

In [47]:
cuts.head()

0     (500.0, 1000.0]
1    (1000.0, 2000.0]
2     (500.0, 1000.0]
3    (1000.0, 2000.0]
4    (1000.0, 2000.0]
Name: DIST, dtype: category
Categories (5, interval[float64]): [(-inf, 200.0] < (200.0, 500.0] < (500.0, 1000.0] < (1000.0, 2000.0] < (2000.0, inf]]

- 정렬된 범주형 Series가 생성됐다.
- 각 범주의 값을 세어보자.

In [49]:
cuts.value_counts()

(500.0, 1000.0]     20659
(200.0, 500.0]      15874
(1000.0, 2000.0]    14186
(2000.0, inf]        4054
(-inf, 200.0]        3719
Name: DIST, dtype: int64

- cuts Series는 그룹화를 위해 사용된다.
- pandas는 원하는 어떤 방식이든 그룹을 형성할 수 있다.
- cuts Series를 groupby 메서드에 전달하고 AIRLINE 열에 value_counts() 메서드를 호출하면 각 거리 그룹별로 분포를 찾을 수 있다.

In [51]:
flights.groupby(cuts)['AIRLINE'].value_counts(normalize=True).round(3).head()

DIST           AIRLINE
(-inf, 200.0]  OO         0.326
               EV         0.289
               MQ         0.211
               DL         0.086
               AA         0.052
Name: AIRLINE, dtype: float64

- OO AIRLINE의 약 33%가 200 마일 미만이다.

- cut 함수는 DIST col의 각 값을 하나의 칸에 매핑한다.
- 각 칸의 한계선 밖에 있는 값은 누락값으로 설정되고 칸에 소속되지 않는다.
- cuts 변수는 5개의 정렬된 범주의 Series다.
- groupby는 쳣ㄴ 변수에 있는 값에 의해 그룹화한다.
- 각 그룹화에 대해 value_counts에 normalize를 True로 설정하여 항공사별 비행 비율을 찾아낸다.

- cuts 변수로 그룹화하면 다른 더 많은 결과를 얻을 수 있다.
- 각 비행거리 그룹마다 25%, 50%,75% 비행 시간을 구할 수 있다.

In [52]:
flights.groupby(cuts)['AIR_TIME'].quantile(q=[.25, .5, .75]).div(60).round(2)

DIST                  
(-inf, 200.0]     0.25    0.43
                  0.50    0.50
                  0.75    0.57
(200.0, 500.0]    0.25    0.77
                  0.50    0.92
                  0.75    1.05
(500.0, 1000.0]   0.25    1.43
                  0.50    1.65
                  0.75    1.92
(1000.0, 2000.0]  0.25    2.50
                  0.50    2.93
                  0.75    3.40
(2000.0, inf]     0.25    4.30
                  0.50    4.70
                  0.75    5.03
Name: AIR_TIME, dtype: float64

- cut 함수를 이용할 때 이 정보를 사용하면 정보가 담긴 문자열 레이블을 생성할 수 있따.
- 이 레이블들로 구간 표기를 바꾼다.
- 안쪽 인덱스 레벨을 컬럼의 col로 변환하는 unstack 함수를 체인시킬 수도 있다.

In [53]:
labels = ['Under an Hour', '1 Hour', '1-2 Hours', '2-4 Hours', '4+ Hours']

In [55]:
cuts2 = pd.cut(flights['DIST'], bins=bins, labels=labels)

In [57]:
cuts2.head()

0    1-2 Hours
1    2-4 Hours
2    1-2 Hours
3    2-4 Hours
4    2-4 Hours
Name: DIST, dtype: category
Categories (5, object): [Under an Hour < 1 Hour < 1-2 Hours < 2-4 Hours < 4+ Hours]

In [58]:
flights.groupby(cuts2)['AIRLINE'].value_counts(normalize=True)\
       .round(3)\
       .unstack()\
       .style.highlight_max(axis=1)

AIRLINE,AA,AS,B6,DL,EV,F9,HA,MQ,NK,OO,UA,US,VX,WN
DIST,,,,,,,,,,,,,,
Under an Hour,0.052,nan,nan,0.086,0.289,nan,nan,0.211,nan,0.326,0.027,nan,nan,0.009
1 Hour,0.071,0.001,0.007,0.189,0.156,0.005,nan,0.1,0.012,0.159,0.062,0.016,0.028,0.194
1-2 Hours,0.144,0.023,0.003,0.206,0.101,0.038,nan,0.051,0.03,0.106,0.131,0.025,0.004,0.138
2-4 Hours,0.264,0.016,0.003,0.165,0.016,0.031,nan,0.003,0.045,0.046,0.199,0.04,0.012,0.16
4+ Hours,0.212,0.012,0.08,0.171,nan,0.004,0.028,nan,0.019,nan,0.289,0.065,0.074,0.046


## 11.도시 간 총 비행 횟수 계산

- 항공 데이터셋에는 출발지와 목적지 공항에 대한 자료가 있다.
- 휴스턴에서 이륙해서 애틀란타에 착률하는 비행편수를 계산하는 것은 아주 간단하다.
- 출발과 도착에 상관없이 두 도시 사이의 총 비행편수를 계산해보자.
- 출발지와 도착지 공항을 알파벳순으로 정렬해 공항 간의 각 조합이 항상 같은 순서로 나타나도록 한다.

In [60]:
flights = pd.read_csv('../data/flights.csv')

In [61]:
flights_ct = flights.groupby(['ORG_AIR', 'DEST_AIR']).size()

In [62]:
flights_ct.head()

ORG_AIR  DEST_AIR
ATL      ABE         31
         ABQ         16
         ABY         19
         ACY          6
         AEX         40
dtype: int64

- 휴스턴(IAH)과 애틀란타(ATL) 사이에 양쪽 방향의 총 비행편수를 선택한다.

In [63]:
flights_ct.loc[('ATL', 'IAH'), ('IAH', 'ATL')]

ORG_AIR  DEST_AIR
ATL      IAH         121
IAH      ATL         148
dtype: int64

- 모든 비행편에 적용할 수 있는 더 효율적이고 자동화된 방법을 찾아보자.
- 각 행에 있는 출발지와 목적지를 독립적으로 알파벳순으로 정렬해보자.

In [65]:
flights_sort = flights[['ORG_AIR', 'DEST_AIR']].apply(sorted, axis=1)

In [66]:
flights_sort.head()

,ORG_AIR,DEST_AIR
0,LAX,SLC
1,DEN,IAD
2,DFW,VPS
3,DCA,DFW
4,LAX,MCI


- 각 row는 독립적으로 정렬됐으므로 col 이름은 정확하지 않다.
- 보다 포괄적인 이름으로 바꾸고 다시 한번 전체 도시 간의 총 비행편수를 찾아보자.

In [68]:
rename_dict = {'ORG_AIR': 'AIR1', 'DEST_AIR': 'AIR2'}

In [70]:
flights_sort = flights_sort.rename(columns=rename_dict)

In [71]:
flights_ct2 = flights_sort.groupby(['AIR1', 'AIR2']).size()

In [73]:
flights_ct2.head()

AIR1  AIR2
ABE   ATL     31
      ORD     24
ABI   DFW     74
ABQ   ATL     16
      DEN     46
dtype: int64

In [75]:
flights_ct2.loc[('ATL', 'IAH')]

269

- flight_count Series는 두 레벨의 MultiIndex를 가진다.
- MultiIndex에서 행을 선택하는 방법 중 하나는 loc 인덱스 연산자에 정확한 레벨 값을 가진 튜플을 전달하는 것이다.

- apply() 메서드를 호출할 때 axis=1로 설정하는 것은 pandas에서 제공하는 연산 중 가장 성능이 저하되는 것 중 하나다.
- pandas의 각 행을 따라가면 연산하는 것은 Numpy로 부터 속도 향상을 받지 못하는 경우가 많다.
- 가능하다면 apply를 사용할 대 axis=1로 설정하는 것은 피하는 것이 좋다.

- NumPy sort 함수를 사용하면 엄청난 속도 향성을 얻을 수 있다.
- 디폴트로 NumPy sort 함수는 각 행을 독립적으로 정렬한다.

In [76]:
data_sorted = np.sort(flights[['ORG_AIR', 'DEST_AIR']])

In [77]:
data_sorted[:10]

array([['LAX', 'SLC'],
       ['DEN', 'IAD'],
       ['DFW', 'VPS'],
       ['DCA', 'DFW'],
       ['LAX', 'MCI'],
       ['IAH', 'SAN'],
       ['DFW', 'MSY'],
       ['PHX', 'SFO'],
       ['ORD', 'STL'],
       ['IAH', 'SJC']], dtype=object)

- 2차원 NumPy 배열이 반환된다.
- NumPy에서는 그룹화가 용이하지 않으므로 DataFrame 생성자를 이용해 새로운 DataFrame을 생성하고, 이 값이 flights_sorted DataFrame과 동일한지 확인해보자.

In [78]:
flights_sort2 = pd.DataFrame(data_sorted, columns=['AIR1', 'AIR2'])

In [79]:
fs_orig = flights_sort.rename(columns={'ORG_AIR': 'AIR1', 'DEST_AIR': 'AIR2'})

In [80]:
flights_sort2.equals(fs_orig)

True

- 시간 차이를 측정해보자.

In [81]:
%%timeit
flights_sort = flights[['ORG_AIR', 'DEST_AIR']].apply(sorted, axis=1)

6.23 s ± 63.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [83]:
%%timeit
data_sorted = np.sort(flights[['ORG_AIR', 'DEST_AIR']])
flights_sort2 = pd.DataFrame(data_sorted, columns=['AIR1', 'AIR2'])

8.18 ms ± 51.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## 12.최장 연속 정시 운행 찾기

- pandas는 직접적으로 항공사별로 총 정시 비행편수와 빙율을 계산할 수 있다.
- 각 항공사별로 출발 공항에 따라 최장 연속 정시 비행같은 계산도 가능하다.

- 샘플 데이터로 연속된 1을 찾는 연습을 해보자.

In [84]:
s = pd.Series([0, 1, 1, 0, 1, 1, 1, 0])

In [85]:
s

0    0
1    1
2    1
3    0
4    1
5    1
6    1
7    0
dtype: int64

In [86]:
s1 = s.cumsum()

In [87]:
s1

0    0
1    1
2    2
3    2
4    3
5    4
6    5
7    5
dtype: int64

In [88]:
s.mul(s1)

0    0
1    1
2    2
3    0
4    3
5    4
6    5
7    0
dtype: int64

In [89]:
s.mul(s1).diff()

0    NaN
1    1.0
2    1.0
3   -2.0
4    3.0
5    1.0
6    1.0
7   -5.0
dtype: float64

In [90]:
s.mul(s1).diff().where(lambda x: x < 0)

0    NaN
1    NaN
2    NaN
3   -2.0
4    NaN
5    NaN
6    NaN
7   -5.0
dtype: float64

In [91]:
s.mul(s1).diff().where(lambda x: x < 0).ffill()

0    NaN
1    NaN
2    NaN
3   -2.0
4   -2.0
5   -2.0
6   -2.0
7   -5.0
dtype: float64

In [92]:
s.mul(s1).diff().where(lambda x: x < 0).ffill().add(s1, fill_value=0)

0    0.0
1    1.0
2    2.0
3    0.0
4    1.0
5    2.0
6    3.0
7    0.0
dtype: float64